In [1]:
import numpy as np
import pandas as pd
import datetime
import pickle
import os

from sklearn.utils import class_weight
from sklearn.model_selection import train_test_split
from keras.models import Sequential, load_model, Model
from keras.layers import Dense, Conv2D, MaxPooling2D, Flatten, Dropout, BatchNormalization, GlobalAveragePooling2D
from keras.callbacks import EarlyStopping, ModelCheckpoint
from keras.optimizers import Adam, SGD
from keras.preprocessing.image import ImageDataGenerator
from keras.regularizers import l2
from keras.utils import to_categorical
from keras.applications import VGG16, VGG19

In [2]:
root_path = '.'

## Google Colab

In [3]:
try:
    from google.colab import drive
    drive.mount('/content/drive')

    root_path = '/content/drive/MyDrive/projects/facial_feeling_detection'
except:
    pass

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


##### Check GPU

In [4]:
try:
    from tensorflow.python.client import device_lib

    print(device_lib.list_local_devices())
except:
    pass

[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 15434369622453649375
xla_global_id: -1
, name: "/device:GPU:0"
device_type: "GPU"
memory_limit: 14391508992
locality {
  bus_id: 1
  links {
  }
}
incarnation: 12986964302416614878
physical_device_desc: "device: 0, name: Tesla T4, pci bus id: 0000:00:04.0, compute capability: 7.5"
xla_global_id: 416903419
]


## Constants

In [5]:
SEED = 42
LABELS = {0: 'angry', 1: 'disgust', 2: 'fear', 3: 'happy', 4: 'neutral', 5: 'sad', 6: 'surprise'}
TRAIN_PATH = root_path + '/data/train/'
TEST_PATH = root_path + '/data/test/'
MODELS_PATH = root_path + '/models/'
SUBMISSIONS_PATH = root_path + '/submissions/'

In [6]:
BATCH_SIZE = 32

learning_rate = .0001 # .01
dropout_conv = .3
dropout_dense = .5

## Data Loading

In [7]:
data = np.load(root_path + '/data/rgb_data.npz')
X_train = data['X_train']
X_test = data['X_test']
y_train = data['y_train']

with open(root_path + '/data/test_ids.pkl', 'rb') as f:
    test_ids = pickle.load(f)

submission_ids = pd.read_csv(root_path + '/data/test_set.csv')

##### Stratification

In [8]:
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=.2, stratify=y_train, random_state=SEED)

##### Generator

In [9]:
datagen = ImageDataGenerator(
    rotation_range=20,
    zoom_range=0.2,
    width_shift_range=0.1,
    height_shift_range=0.1,
    horizontal_flip=True
)

In [10]:
datagen.fit(X_train)

In [11]:
train_generator = datagen.flow(X_train, y_train, batch_size=BATCH_SIZE)

##### Weights

In [12]:
class_weights = class_weight.compute_class_weight('balanced', classes=np.unique(y_train), y=y_train)
class_weight_dict = {label: weight for label, weight in zip(np.unique(y_train), class_weights)}

##### To Categorical

In [13]:
# y_train = to_categorical(y_train, num_classes=7)

## Model

In [14]:
vgg19 = VGG19(weights='imagenet', include_top=False, input_shape=(48, 48, 3))

# print(vgg19.summary())

model = Model(
    inputs=vgg19.input,
    outputs=Dense(7, activation='softmax')(
        GlobalAveragePooling2D()(
            vgg19.layers[-2].output)))

# print(model.summary())

In [15]:
model.compile(
    optimizer=Adam(learning_rate=learning_rate),
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

In [16]:
callback_path = MODELS_PATH + 'callback.h5'

earlystop = EarlyStopping(patience=10)
mcheckpoint = ModelCheckpoint(
    callback_path,
    monitor='val_accuracy',
    save_best_only=True
)

In [17]:
history = model.fit(
    train_generator,
    # X_train, y_train,
    validation_data=(X_val, y_val),
    epochs=20,
    steps_per_epoch=len(X_train) / BATCH_SIZE,
    batch_size=BATCH_SIZE,
    # class_weight=class_weight_dict,
    callbacks=[earlystop, mcheckpoint],
    use_multiprocessing=True
)

Epoch 1/20
720/720 [==============================] - 52s 66ms/step - loss: 1.4994 - accuracy: 0.4123 - val_loss: 1.3106 - val_accuracy: 0.4952
Epoch 2/20
720/720 [==============================] - 44s 61ms/step - loss: 1.2888 - accuracy: 0.5033 - val_loss: 1.1885 - val_accuracy: 0.5493
Epoch 3/20
720/720 [==============================] - 48s 66ms/step - loss: 1.2151 - accuracy: 0.5312 - val_loss: 1.0939 - val_accuracy: 0.5912
Epoch 4/20
720/720 [==============================] - 46s 63ms/step - loss: 1.1604 - accuracy: 0.5601 - val_loss: 1.0808 - val_accuracy: 0.5946
Epoch 5/20
720/720 [==============================] - 45s 62ms/step - loss: 1.1175 - accuracy: 0.5766 - val_loss: 1.0768 - val_accuracy: 0.5920
Epoch 6/20
720/720 [==============================] - 44s 61ms/step - loss: 1.0782 - accuracy: 0.5932 - val_loss: 1.0769 - val_accuracy: 0.5929
Epoch 7/20
720/720 [==============================] - 44s 61ms/step - loss: 1.0428 - accuracy: 0.6067 - val_loss: 1.0217 - val_accuracy:

In [18]:
max_val_accuracy = max(history.history['val_accuracy'])
date = datetime.datetime.now().strftime('%y%m%d%H%M%S')
model_path = f'{MODELS_PATH}model_val_acc_{max_val_accuracy:.4f}_{date}.h5'

os.rename(callback_path, model_path)

## Prediction/Submission

In [19]:
# model_path = MODELS_PATH + 'model_val_acc_0.4901_230224172910.h5'

model_last = load_model(model_path)

In [20]:
y_pred = model_last.predict(X_test)
predictions = [np.argmax(pred) for pred in y_pred]

221/221 [==============================] - 4s 16ms/step


In [21]:
submission_pred = pd.DataFrame(test_ids, columns=['id_img']).astype(int)
submission_pred['label'] = [LABELS[pred] for pred in predictions]
submission_pred = pd.merge(submission_ids, submission_pred)

In [22]:
submission_path = model_path.replace(MODELS_PATH, SUBMISSIONS_PATH).replace('model_', 'submission_').replace('.h5', '.csv')
submission_pred.to_csv(submission_path, index=False)